In [57]:
!pip install pandas numpy
!pip install loguru

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import json
import numpy as np

data = pd.read_csv("normalized_data.csv")

In [2]:
from loguru import logger

logger.add("API_Errors.log", level = "INFO", format = "{time}:{level}:{message}")

1

In [3]:
import requests

url = "https://smarthome.uni-regensburg.de/naehrwertrechner/api/1.0/recipe_info_optifast"

def get_nut(prompt: str) -> dict | None:
    payload = { "recipe": prompt }
    headers = {"content-type": "application/json"}
    response = requests.post(url, json = payload, headers = headers, verify=False)
    
    if response.ok:
        return response.json()
    else:
        return None

In [4]:
import json

def calc_nut(amount: int, unit: str, ingr: str) -> str:
    """
        calculate the nutrition value using ingredient name and quantity.
        If the output is "Nicht Erkannt" then this function would returns np.NaN
        And log the error in a seperate file when an error occurs at the API
    """
    prompt = f"{amount}{unit} {ingr}"
    res = get_nut(prompt)
    if not res or res["detailed_info"][0][0]["erkannteZutat"] == "Nicht erkannt":
        return np.nan
    else:
        return json.dumps(res)

In [5]:
data['nutrition'] = data.apply(
    lambda x: calc_nut( x["norm_value"], x["norm_unit"], x["ingredient"]),
    axis = 1,
)

In [7]:
data.to_csv("nutrition_data.csv")

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 910 entries, 0 to 909
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0.1       910 non-null    int64  
 1   Unnamed: 0         910 non-null    int64  
 2   ingredient         910 non-null    object 
 3   amount             910 non-null    object 
 4   ingr_annotation    910 non-null    object 
 5   amount_annotation  910 non-null    object 
 6   norm_value         910 non-null    float64
 7   norm_unit          910 non-null    object 
 8   nutrition          601 non-null    object 
dtypes: float64(1), int64(2), object(6)
memory usage: 64.1+ KB
